In [18]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score 
from prettytable import PrettyTable
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import MinMaxScaler

In [19]:
df = pd.read_csv('../input/us-accidents/US_Accidents_Dec21_updated.csv')

In [20]:
df.dropna(inplace = True)
df=df[df.Severity != 0]
df.drop(columns = ['ID'], inplace = True)
df.drop(columns = ['Description'], inplace = True)
categorical_columns = df.select_dtypes(include=['object','bool','datetime64[ns]']).columns
categorical_columns
from sklearn import preprocessing

for i in categorical_columns:
    en = preprocessing.LabelEncoder()
    df[i]= en.fit_transform(df[i])

In [21]:
target = 'Severity'
y = df[target]
X = df.drop(target, axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=21, stratify=y)

In [22]:
scaler = MinMaxScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)

multinom = MultinomialNB(fit_prior = True,alpha =  0.0054864954984995)
import time
start = time.time()
best_model= multinom.fit(X_train,y_train)
print("Total time: ", time.time() - start, "seconds")
preds = best_model.predict(X_test)
accuracy = accuracy_score(y_test, preds)
print(accuracy)

Total time:  0.23065781593322754 seconds
0.9263664504091931


In [23]:
import xgboost as xgb
from sklearn.ensemble import AdaBoostClassifier
from sklearn.inspection import permutation_importance
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import learning_curve
from sklearn.metrics import classification_report, accuracy_score


In [24]:
from sklearn.svm import SVC
from sklearn.model_selection import KFold
seed = 4353
svc = SVC(random_state=seed)

In [ ]:
# using KFold cross validation technique
kf=  KFold(n_splits=5, random_state=seed,shuffle=True)

# Hyperparametric tuning using grid search
param_grid = [{'kernel':['rbf'],
              'gamma':[1e-3, 1e-4],
              'C':[1, 10, 100, 1000]},
             {'kernel':['linear'],
             'C':[1, 10, 100, 1000]}]

grid = GridSearchCV(estimator=svc, param_grid=param_grid, scoring='accuracy', cv=kf)
grid.fit(X_train, y_train)

print('Estimator: ', grid.best_estimator_)
print('Best params : \n', grid.best_params_)
print('Output Classes: ', grid.classes_)
print('Training Accuracy: ', grid.best_score_)

In [ ]:
# predictions

predictions = grid.predict(X_test)

print(classification_report(y_test, predictions))

svc_f1 = round(f1_score(y_test, predictions, average='weighted'), 3)
svc_accuracy = round((accuracy_score(y_test, predictions)*100), 2)

print("Accuracy : " , svc_accuracy , " %")